In [1]:
import cv2
from datetime import datetime
import itertools as it
import numpy as np
import random
from typing import Tuple

class mx:
    class context:
        class Context:
            pass
    def cpu():
        pass
    class nd:
        class NDArray:
            pass
        def array(array, ctx):
            return array

class Timer:
    def __init__(self, message: str):
        self.__message = message

    def __enter__(self):
        self.__start_at = datetime.now()

    def __exit__(self, *args):
        print(f'{self.__message} time:', (datetime.now() - self.__start_at).total_seconds(), 'seconds')

In [2]:
max_seq_len = 80
alphabet = list(map(chr, range(ord('A'), ord('Z') + 1))) + list("'-")
alphabet_dict = {alphabet[i]: i for i in range(len(alphabet))}
alphabet_with_spaces = alphabet + round(len(alphabet) / 5) * [' ']
font_face = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1.5
thickness = 2
width, height = 400, 60

def get_text_size(text: str, font_face: int, font_scale: int, thickness: int) -> Tuple[int, int]:
    text_size, _ = cv2.getTextSize(text, font_face, font_scale, thickness)
    return text_size[0] + thickness, text_size[1] + thickness

def put_text(image: np.ndarray, text: str, x: int, y: int) -> None:
    text_size = get_text_size(text, font_face, font_scale, thickness)
    cv2.putText(image, text, (x, y + text_size[1]), font_face, font_scale, (255,), thickness, cv2.LINE_AA)

def make_data(n: int, ctx: mx.context.Context) -> Tuple[mx.nd.NDArray, mx.nd.NDArray]:
    def fn(_):
        letters = None
        while letters is None or len([c for c in letters if c != ' ']) < 2:
            letters = random.choices(alphabet_with_spaces, k=10)
        s = ''.join(letters)
        image = np.zeros((height, width), dtype=np.uint8)
        put_text(image, s, random.randrange(5, 35), 5)
        g = (alphabet_dict[c] for c in letters if c != ' ')
        g = it.chain(g, it.repeat(-1, max_seq_len))
        return image[np.newaxis, :, :] / 255.0, list(g)[:max_seq_len]
    x, y = (mx.nd.array(np.stack(v, axis=0), ctx=ctx) for v in zip(*map(fn, range(n))))
    return x, y

with Timer('make_data'):
    inputs, outputs = make_data(999, mx.cpu())
print(inputs.min(), inputs.max(), inputs.dtype, inputs.shape, outputs.min(), outputs.max(), outputs.dtype, outputs.shape)
image = np.vstack(inputs[:9])
image = image.reshape(-1, image.shape[-1])
print(image.shape)
cv2.imshow('tesst', image)
cv2.waitKey()
cv2.destroyWindow('tesst')
np.stack(inputs, axis=0).shape, list(map(len, outputs[:9]))

make_data time: 0.263294 seconds
0.0 1.0 float64 (999, 1, 60, 400) -1 27 int32 (999, 80)
(540, 400)


((999, 1, 60, 400), [80, 80, 80, 80, 80, 80, 80, 80, 80])